# Learning about Convolutions

Learning about convolutions step-by-step.

In [1]:
import torch
import torch.nn as nn

## 1D Convolutions

In [26]:
x = torch.tensor([[1., -2., 3., -4.],
                  [1., -2., 3., -4.],
                  [1., -2., 3., -4.]])

In [27]:
x.shape

torch.Size([3, 4])

In [29]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 3, kernel_size = 2)
conv(x)

tensor([[ 0.1801,  0.0205,  0.1134],
        [ 0.4970, -0.3299,  0.7912],
        [ 0.7282, -0.6791,  1.4015]], grad_fn=<SqueezeBackward1>)

In [30]:
conv(x).shape

torch.Size([3, 3])

In [31]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 2, kernel_size = 2)
conv(x)

tensor([[-0.0924,  0.9214, -0.2646],
        [-0.2516,  0.2981, -0.8774]], grad_fn=<SqueezeBackward1>)

In [32]:
conv(x).shape

torch.Size([2, 3])

In [33]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 3, kernel_size = 3)
conv(x)

tensor([[-0.1639,  0.3818],
        [-1.1151,  1.7506],
        [ 0.7499, -0.9797]], grad_fn=<SqueezeBackward1>)

In [34]:
conv(x).shape

torch.Size([3, 2])